# An example to show how to execute a TAP query with authentication

__DESCRIPTION__

Script to execute a query using pyvo library to obtain data through archive TAP service with authentication

@authors Tamara Civera & Javier Hernandez (CEFCA)

In [ ]:
# Imports
import pyvo.dal 
from pyvo.auth import authsession, securitymethods
import requests
import getpass

In [ ]:
# To avoid warnings

import warnings
warnings.simplefilter("ignore")

In [ ]:
archive_url = 'https://archive.cefca.es/catalogues'
survey_dr = 'jplus-dr2'
url_tap_service = '{0}/vo/tap/{1}'.format(archive_url, survey_dr)

user = input("Username:")
pwd = getpass.getpass("Password:") 

archive_login_url = '{0}/login'.format(archive_url)
login_args = {'login': user, 'password': pwd, 'submit': 'Sign+In'} 
login_header = {'Content-type': 'application/x-www-form-urlencoded', 'Accept': 'text/plain'} 

__Starting Session__

In [ ]:
pyvo.dal.tap.s = requests.Session()
response = pyvo.dal.tap.s.post(archive_login_url, data=login_args, headers=login_header)
response.raise_for_status() 
auth = authsession.AuthSession()
auth.credentials.set(securitymethods.ANONYMOUS, pyvo.dal.tap.s) 


__Synchronous mode__

In [ ]:
service = pyvo.dal.TAPService(url_tap_service, auth)
resultset = service.run_sync("SELECT TOP 5 * FROM jplus.TileImage")
resultset.to_table()

__Asynchronous mode: Example 1__

In [ ]:
# Execute the query in asynchronous mode (simplest way): Works in VOTABLE format

service = pyvo.dal.TAPService(url_tap_service, auth)
resultset = service.run_async("SELECT TOP 5 * FROM jplus.TileImage")
resultset.to_table()

__Asynchronous mode: Example 2__

In [ ]:
# Execute the query in asynchronous mode (working directly with jobs): Works in VOTABLE format

service = pyvo.dal.TAPService(url_tap_service, auth)
job = service.submit_job("SELECT TOP 5 * FROM jplus.TileImage")
job.run()
job.wait()
job.raise_if_error()
resultset = job.fetch_result()
print(resultset)
job.delete()

__Asynchronous mode: Example 3__

In [ ]:
# Execute the query in asynchronous mode (working directly with jobs): Works in FITS format

service = pyvo.dal.TAPService(url_tap_service, auth)
job = service.submit_job("SELECT TOP 10 * FROM jplus.TileImage", maxrec=300, format="FITS")
dest_name = "data/TileImageData.fits"
try:
    job.run()
    job.wait()
    job.raise_if_error()
    with open(dest_name, "wb") as dest:        
        session = requests.Session()
        response = session.get(job.result_uri, auth=(user, pwd), verify=False)
        response.raise_for_status()
        dest.write(response.content)
finally:
    job.delete()

__Asynchronous mode: Example 4__

In [ ]:
# Asyncronous mode with jobs: Works in CSV format

service = pyvo.dal.TAPService(url_tap_service, auth)
job = service.submit_job("SELECT * FROM jplus.TileImage", maxrec=300, format="CSV")
dest_name = "data/TileImageData.csv"
try:
    job.run()
    job.wait()
    job.raise_if_error()
    with open(dest_name, "w") as dest:        
        session = requests.Session()
        response = session.get(job.result_uri, auth=(user, pwd), verify=False)
        response.raise_for_status()
        dest.write(response.text)
finally:
    job.delete()